# LLM - Visual captioning

## Imagen imagetext
- https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning

In [1]:

import vertexai
from google.cloud import bigquery
from tools import *
from bq import save_bq


PROJECT_ID = "ml-demo-384110"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")
client = bigquery.Client(project=PROJECT_ID)
bucket = "ml-demo-384110-bucket"

%load_ext google.cloud.bigquery

In [3]:
# @title Parameters
LOCATION = "us-central1" #@param {type:"string"}
#PROJECT_NUMBER = 
#PROJECT_ID = "" #@param {type:"string"}

PROJECT_ID = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_ID)"

if len(PROJECT_ID) > 1:
    PROJECT_ID = PROJECT_ID[1]
else:
    PROJECT_ID = PROJECT_ID[0]
print(PROJECT_ID)

PROJECT_NUMBER = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_NUMBER)"
if len(PROJECT_NUMBER) > 1:
    PROJECT_NUMBER = PROJECT_NUMBER[1]
else:
    PROJECT_NUMBER = PROJECT_NUMBER[0]

print(PROJECT_NUMBER)

TOKEN_ENDPOINT_URI = 'https://www.googleapis.com/oauth2/v3/tokeninfo?access_token='

IMAGEGEN_MODEL = "imagegeneration@002" #models available: imagegeneration@001 and imagegeneration@002
ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGEGEN_MODEL}'

IMAGE_TEXT_MODEL = "imagetext"
IMAGEN_ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGE_TEXT_MODEL}'









ml-demo-384110
1008225662928


In [4]:
# @title Setup

import json
from time import sleep
from ratelimit import limits, sleep_and_retry
import requests
import time

from IPython import display
import base64

"""
Get access token
"""
def get_token():
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get(TOKEN_ENDPOINT_URI + gcloud_token[0]).json()
    print("Access Token is generated for ", gcloud_tokeninfo['email'])
    ACCESS_TOKEN = gcloud_token[0]
    return ACCESS_TOKEN


ACCESS_TOKEN = get_token()

# @title image Generation Model
def generate_image(
    prompt: str,
    sampleImageSize: str,
    sampleCount: int,
    seed: int
    ):

        

    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    data = {"instances": [{"prompt": prompt}],
            "parameters": {"sampleImageSize": sampleImageSize,"sampleCount": sampleCount, "seed": seed}}

    response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{ENDPOINT_URL}:predict', data=json.dumps(data), headers=headers)
    return response

###
# Visual Question Answering (VQA) to get image information
##
@sleep_and_retry
@limits(calls=400, period=60)
def visual_qa(
    prompt: str,
    sampleCount: int,
    encoded_str: str
    ):
    global ACCESS_TOKEN
    
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {
        "instances": [
            {
            "prompt": prompt,
            "image": {
                "bytesBase64Encoded": encoded_str
            }
            }
        ],
        "parameters": {
            "sampleCount": sampleCount
            }
        }


    status_code = 0
    while status_code != 200:
        response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{IMAGEN_ENDPOINT_URL}:predict', 
            data=json.dumps(data), headers=headers)
        
        if response.status_code == 200:
            return response
        
        if response.status_code == 401:
            ACCESS_TOKEN = get_token()

        if response.status_code == 500:
            print("ERROR 500 - pass error.")
            return response
        
        print(f"ERROR status_code: {response.status_code} - {response.text}") 
        status_code = response.status_code
        sleep(2)

    return response

###
#Visual captioning is available in the following languages:
# English (en)
# French (fr)
# German (de)
# Italian (it)
# Spanish (es)
##
@sleep_and_retry
@limits(calls=400, period=60)
def visual_captioning(
    sampleCount: int,
    language: str, 
    # English (en), French (fr), German (de), Italian (it), Spanish (es)
    encoded_str: str
    ):
    global ACCESS_TOKEN

    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {
        "instances": [
            {
            "image": {
                "bytesBase64Encoded": encoded_str
                }
            }
        ],
        "parameters": {
            "sampleCount": sampleCount,
            "language": language
            }
        }

    status_code = 0
    while status_code != 200:
        response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{IMAGEN_ENDPOINT_URL}:predict', 
            data=json.dumps(data), headers=headers)
        
        if response.status_code == 200:
            return response
        
        print(f"ERROR status_code: {response.status_code} - {response.text}") 
        if response.status_code == 401:
            ACCESS_TOKEN = get_token()

        if response.status_code == 500:
            print("ERROR 500 - pass error.")
            return response

        print(f"ERROR status_code: {response.status_code} - {response.text}") 
        status_code = response.status_code
        
        sleep(2)


Access Token is generated for  admin@julienmiquel.altostrat.com


In [5]:
%%bigquery df
      SELECT distinct uri, Id,
      FROM `immo_us_enriched.image_cloudvision_labels_1k` 
      where uri not in (
            SELECT uri FROM `ml-demo-384110.immo_us.visual_captioning` 
            where caption_fr != "" and caption_en != "" 
            )
            #limit 100



Query is running:   0%|          |

Downloading:   0%|          |

In [55]:
df.describe()


,uri,Id
count,864,864
unique,864,64
top,gs://ml-demo-384110-bucket/dataset/immo/images...,apimo-7181337
freq,1,26


In [6]:
df.describe()


,uri,Id
count,425,425
unique,425,41
top,gs://ml-demo-384110-bucket/dataset/immo/images...,safti-1-807163
freq,1,20


In [7]:

from tools import *

def add_images_caption_to_df(df, language = "en", bucket_name = "ml-demo-384110-bucket"):

    df[f'caption_{language}' ] = df.uri.map(lambda uri_name:                                    
                                   
                                   print_text_response(visual_captioning( 1, language,str(read_b64_from_gcs(bucket, uri_name.replace(f"gs://{bucket_name}/", "")), 'UTF-8'))
                                  )
                                  )
    
    return df


def add_images_visual_caption_to_df(df, prompt, name, number_of_answer, language = "en", bucket_name = "ml-demo-384110-bucket"):

    df[f'caption_{name}' ] = df.uri.map(lambda uri_name: 
                                   print_text_response(visual_qa(prompt, number_of_answer, language,str(read_b64_from_gcs(bucket, uri_name.replace(f"gs://{bucket_name}/", "")), 'UTF-8'))
                                  )
                                  )
    
    return df





In [8]:

BQ_DATASET_NAME = 'immo_us' 
BQ_TABLE_NAME = 'visual_captioning'
project_table = PROJECT_ID + "." +BQ_DATASET_NAME +"." + BQ_TABLE_NAME


## Process description


In [9]:
BQ_DATASET_NAME = 'immo_us' 
BQ_TABLE_NAME = 'visual_captioning'
project_table = PROJECT_ID + "." +BQ_DATASET_NAME +"." + BQ_TABLE_NAME


def process_description(df):

    for id in df.Id.drop_duplicates().tolist() :
        print(id)
        df_vqa = df[df["Id"] == id]
        
        df_vqa = add_images_caption_to_df(df_vqa, "en")
        df_vqa = add_images_caption_to_df(df_vqa, "fr")

        df_vqa["update_time"] = update_time()

        print(f"SAVE index chunk : {id}")
        save_bq(df_vqa, table_id=project_table, project_id=PROJECT_ID)

process_description(df)

ag340369-381741688
a floor plan of a house with three bedrooms and a spiral staircase .


/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_22858/4168347006.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:


ERROR status_code: 500 - {
  "error": {
    "code": 500,
    "message": "Internal error encountered.",
    "status": "INTERNAL"
  }
}

ERROR 500 - pass error.
<Response [500]>
SAVE index chunk : ag340369-381741688
save_bq


/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_22858/4168347006.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:
/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_22858/155183685.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vqa["update_time"] = update_time()
100%|██████████| 1/1 [00:00<00:00, 10645.44it/s]


ag753296-384794585
a room with a chair and a desk and the word hosman on the bottom


...

un long couloir avec un tapis roulant et des pots de fleurs avec le logo progress-immo
SAVE index chunk : apimo-8222658
save_bq


/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_22858/4168347006.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:
/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_22858/155183685.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vqa["update_time"] = update_time()
100%|██████████| 1/1 [00:00<00:00, 9619.96it/s]


apimo-82421909
a dining room with a table and chairs and a sign that says morisse immobilier


/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_22858/4168347006.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:


KeyboardInterrupt: 

In [ ]:
BQ_DATASET_NAME = 'immo_us' 
BQ_TABLE_NAME = 'visual_captioning_attributes'
project_table = PROJECT_ID + "." +BQ_DATASET_NAME +"." + BQ_TABLE_NAME

def process_attributes(df):
    
    for id in df.Id.drop_duplicates().tolist() :

        print(id)
        df_vqa = df[df["Id"] == id]

        df_vqa = add_images_visual_caption_to_df(df_vqa, "what is the room ?", "location", 1, "en")
        df_vqa = add_images_visual_caption_to_df(df_vqa, "floor material", "floor", 1, "en")
        df_vqa = add_images_visual_caption_to_df(df_vqa, "wall material", "wall", 1, "en")

        df_vqa = add_images_visual_caption_to_df(df_vqa, "describe", "describe", 3, "en")
        df_vqa = add_images_visual_caption_to_df(df_vqa, "describe windows", "windows", 3, "en")
        df_vqa = add_images_visual_caption_to_df(df_vqa, "text", "text", 3, "en")
        df_vqa = add_images_visual_caption_to_df(df_vqa, "Level of finish of the place", "finish_level", 3, "en")

        df_vqa["update_time"] = update_time()

        print(f"SAVE index chunk : {id}")
        save_bq(df_vqa, 
                table_id=PROJECT_ID + "." +BQ_DATASET_NAME +".visual_captioning_tags" , 
                project_id=PROJECT_ID)

process_attributes(df)

# Analys results

## Translate result to analyse quality in different language.

In [12]:
%%bigquery

create or replace table   ml-demo-384110.immo_us_enriched.visual_captioning_translated as (
  SELECT uri,
  text_content as caption_fr,  
  string(ml_translate_result.translations[0].translated_text) as caption_fr_translated_en, 
  caption_en,  
  string(ml_translate_result.translations[0].detected_language_code) as detected_language_code,
  ml_translate_status

  from ML.TRANSLATE(
  model `ml-demo-384110.immo_us_enriched.bqml-cloud-ai-translate`,
  (SELECT distinct caption_fr AS text_content, caption_en from `ml-demo-384110.immo_us.visual_captioning` where caption_fr != "" ),
  STRUCT('translate_text' AS translate_mode, 'en' AS target_language_code)
)
);

Query is running:   0%|          |

""


## Rouge


In [15]:
%%bigquery df

select * from  ml-demo-384110.immo_us_enriched.visual_captioning_translated where ml_translate_status = ""



Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
from rouge import Rouge 
import json



def calculate_rouge(candidate, reference):
    rouge = Rouge()
    '''
    candidate, reference: generated and ground-truth sentences
    '''
    scores = rouge.get_scores(candidate, reference)
    return scores[0]
    
    return json.dumps(scores)

In [7]:
hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

scores = calculate_rouge(hypothesis, reference)
scores['rouge-1']




{'r': 0.42857142857142855, 'p': 0.5833333333333334, 'f': 0.49411764217577864}